# Loading data

In [1]:
import glob
import os
import numpy as np
import math
import pandas as pd
from IPython.display import display
import pickle
from sklearn.externals import joblib

#Reach out of IMU
IMU_RO_file_path = r'C:\Users\NCTU\Desktop\WhoTakeWhat\data\IMU\forRO\training\ReachOut'                     # use your path
IMU_RO_all_files = glob.glob(os.path.join(IMU_RO_file_path, "*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent

IMU_RO_df_list = []

for index, f in enumerate(IMU_RO_all_files):
    df = pd.read_csv(f, header=None)
    df.columns = ['earx', 'eary', 'earz', 'accx', 'accy', 'accz', 'gyrox', 'gyroy', 'gyroz', 'timestamp', 'sampleNum']
    IMU_RO_df_list.append(df)
    

#Other of IMU
IMU_Other_file_path = r'C:\Users\NCTU\Desktop\WhoTakeWhat\data\IMU\forRO\training\Other'                     # use your path
IMU_Other_all_files = glob.glob(os.path.join(IMU_Other_file_path, "*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent

IMU_Other_df_list = []

for index, f in enumerate(IMU_Other_all_files):
    df = pd.read_csv(f, header=None)
    df.columns = ['earx', 'eary', 'earz', 'accx', 'accy', 'accz', 'gyrox', 'gyroy', 'gyroz', 'timestamp']
    IMU_Other_df_list.append(df)
#============================================== 

config_file_path = './config/'

### Reach out data

取出各個reach out 的sample段 to list that each of element is np.array

In [2]:
IMU_RO_sampleAll = []

for index, IMU_RO_raw_pd in enumerate(IMU_RO_df_list):
    sampleCount = [x for x in np.unique(IMU_RO_raw_pd['sampleNum'].values) if not math.isnan(x)]
    for count in sampleCount:
        IMU_RO_sampleAll.append(IMU_RO_raw_pd.loc[IMU_RO_raw_pd['sampleNum']==count, ['accx', 'accy', 'accz', 'gyrox', 'gyroy', 'gyroz']].values)

print("Training IMU data of reaching out have %d sample" % (len(IMU_RO_sampleAll)))

Training IMU data of reaching out have 45 sample


# Estimate windows size

In [3]:
# How much samples in one sec (Must need even)
sampleRate_IMU_acc = 100

# Config file
config_file = {}
config_file['sampling_rate'] = sampleRate_IMU_acc

取最平均值↓↓↓↓↓↓↓↓↓

In [4]:
windowSize_IMU_sa = 0
lenTemp = []
for IMU_RO_sample in IMU_RO_sampleAll:
    lenTemp.append(len(IMU_RO_sample))

windowSize_IMU_sa = np.mean(lenTemp)
windowSize_IMU = -(-(windowSize_IMU_sa)//sampleRate_IMU_acc)
print ('IMU window: ', windowSize_IMU)

        
#Convert to sec
# windowSize_sec = max(windowSize_IMU)
windowSize_sec = windowSize_IMU

windowSize_sec = int(windowSize_sec)

config_file['windows_size'] = windowSize_sec

print ('Windows Size = %d' % windowSize_sec)

IMU window:  3.0
Windows Size = 3


In [5]:
pickle.dump(config_file, open((config_file_path + 'config.pkl'), "wb"))

取最大值↓↓↓↓↓↓↓↓↓

In [6]:
# #IMU
# windowSize_IMU_sa = 0
# for IMU_RO_sample in IMU_RO_sampleAll:
#     if windowSize_IMU_sa < IMU_RO_sample.shape[0]:
#         windowSize_IMU_sa = IMU_RO_sample.shape[0]
        
# windowSize_IMU = -(-(windowSize_IMU_sa)//sampleRate_IMU)
# print ('IMU window: ', windowSize_IMU)

# #Kinect    
# windowSize_Kinect_sa = 0
# for Kinect_RO_sample in Kinect_RO_sampleAll:
#     if windowSize_Kinect_sa < Kinect_RO_sample.shape[0]:
#         windowSize_Kinect_sa = Kinect_RO_sample.shape[0]

# windowSize_Kinect = -(-(windowSize_Kinect_sa)//sampleRate_Kinect)
# print ('Kinect window: ', windowSize_Kinect)
        
# #Convert to sec
# windowSize_sec = max(windowSize_IMU, windowSize_Kinect)

# windowSize_sec = int(windowSize_sec)

# # windowSize_sec = 3
# print ('Windows Size = %d' % windowSize_sec)

# Cut other data by windows size (overlapping half)

In [7]:
#IMU other data
IMU_Other_sampleAll = []
for index, IMU_Other_raw_pd in enumerate(IMU_Other_df_list):
    ranges = list(range(0, IMU_Other_raw_pd.shape[0], windowSize_sec*sampleRate_IMU_acc//2))
    for start in ranges:
        end = start+windowSize_sec*sampleRate_IMU_acc
        if end >= len(IMU_Other_raw_pd):
            continue
        IMU_Other_sampleAll.append(IMU_Other_raw_pd.loc[start:end-1, ['accx', 'accy', 'accz', 'gyrox', 'gyroy', 'gyroz']].values)

print("Training IMU data of Other have %d sample" % len(IMU_Other_sampleAll))

Training IMU data of Other have 481 sample


# Preprocessing

# Feature extraction

### Feature of IMU

Acc

In [8]:
from extract_feature_IMU import extract_feature_IMU

Gyro

In [9]:
# def extract_feature_gyro_IMU(IMU_sample): #input np.array shape = (N, 3)

#     ##Mean (1, 3)
#     IMU_mean = np.mean(IMU_sample, axis=0) 
    
#     ##Skewness
#     IMU_skew = skew(IMU_sample, axis=0)
    
#     ##Kurtosis
#     IMU_kurtosis = kurtosis(IMU_sample, axis=0)
    
#     ##Mean Magnitude (1,)
#     summ = 0.0
#     for IMU_sample_sample in IMU_sample:
#         tempSqur = 0.0
#         for i in range(0, len(IMU_sample_sample)):
#             tempSqur += IMU_sample_sample[i]**2
#         summ = tempSqur**0.5
#     IMU_MM = (summ/IMU_sample.shape[0])

#     ##Variance (1, 3)
#     IMU_var = np.var(IMU_sample, axis=0) 

#     ##Median (1, 3)
#     IMU_median = np.median(IMU_sample, axis=0)
    
#     #-------------------------------------------
#     #
    
#     #-------------------------------------------
#     #Time feature

#     ##Mean of 1 sec (1, 3*windowSize)...
#     #feature count is windowsSize/sec. 
#     #Each of mean of reaching out are 100 samples.
#     oneSample = []
#     for start in range(0, windowSize_sec*sampleRate_IMU_gyro, sampleRate_IMU_gyro):
#         end = start + sampleRate_IMU_gyro
#         tempSamples = IMU_sample[start:end]
#         oneSample.extend(np.mean(tempSamples, axis=0).tolist())
#     IMU_meanOf1sec = oneSample

#     #combine together
#     features = []
#     features.extend(IMU_mean.tolist())
#     features.extend(IMU_skew.tolist())
#     features.extend(IMU_kurtosis.tolist())
#     features.append(IMU_MM)
#     features.extend(IMU_var.tolist())
#     features.extend(IMU_median.tolist())
#     features.extend(IMU_meanOf1sec)
    
#     return features

## 補足data長度到windows size(前後補)&Split data into Training, Validation and Testing(6/2/2) & Standard

#### IMU

In [10]:
X_IMU = []
y_IMU = []

#add RO into X_IMU & y_IMU
for IMU_RO_sample in IMU_RO_sampleAll:
    lengthOflack = sampleRate_IMU_acc*windowSize_sec-len(IMU_RO_sample)
    if lengthOflack > 0:
#         IMU_RO_sample_filltoprior = np.concatenate( 
#                                   ([IMU_RO_sample[0].tolist()] * lengthOflack, IMU_RO_sample),
#                                   axis=0)
#         IMU_RO_sample_filltoPandR = np.concatenate(
#                                   ([IMU_RO_sample[0].tolist()] * (lengthOflack//2), IMU_RO_sample, [IMU_RO_sample[-1].tolist()] * (-(-lengthOflack//2))), 
#                                   axis=0)
#         IMU_RO_sample_filltorear = np.concatenate(
#                                   (IMU_RO_sample, [IMU_RO_sample[-1].tolist()] * lengthOflack), 
#                                   axis=0)

        IMU_RO_sample_filltoprior = np.concatenate( 
                                  ([[0, 0, 0, 0, 0, 0]] * lengthOflack, IMU_RO_sample),
                                  axis=0)
        IMU_RO_sample_filltoPandR = np.concatenate(
                                  ([[0, 0, 0, 0, 0, 0]] * (lengthOflack//2), IMU_RO_sample, [[0, 0, 0, 0, 0, 0]] * (-(-lengthOflack//2))), 
                                  axis=0)
        IMU_RO_sample_filltorear = np.concatenate(
                                  (IMU_RO_sample, [[0, 0, 0, 0, 0, 0]] * lengthOflack), 
                                  axis=0)
    else:
        IMU_RO_sample_filltoprior = IMU_RO_sample
        IMU_RO_sample_filltoPandR = IMU_RO_sample
        IMU_RO_sample_filltorear = IMU_RO_sample


    features_prior = extract_feature_IMU(IMU_RO_sample_filltoprior)
    features_PandR = extract_feature_IMU(IMU_RO_sample_filltoPandR)
    features_rear = extract_feature_IMU(IMU_RO_sample_filltorear)

    #add into X_IMU
    X_IMU.append(features_prior)
    y_IMU.append(1)
    X_IMU.append(features_PandR)
    y_IMU.append(1)
    X_IMU.append(features_rear)
    y_IMU.append(1)
    

#add Other into X_IMU & y_IMU
for IMU_Other_sample in IMU_Other_sampleAll:
    features = extract_feature_IMU(IMU_Other_sample)
    #add into X
    X_IMU.append(features)
    y_IMU.append(0)
    

##Split X_IMU, y_IMU into training and testing(7/3)
from sklearn.model_selection import train_test_split

# X_IMU_train, X_IMU_test, y_IMU_train, y_IMU_test = train_test_split(X_IMU, y_IMU, test_size=0.2, random_state=0)
# X_IMU_train, X_IMU_val, y_IMU_train, y_IMU_val = train_test_split(X_IMU_train, y_IMU_train, test_size=0.25, random_state=0)
X_IMU_train, X_IMU_val, y_IMU_train, y_IMU_val = train_test_split(X_IMU, y_IMU, test_size=0.3, random_state=0)


#Standard
from sklearn.preprocessing import StandardScaler

sc_IMU = StandardScaler()
sc_IMU.fit(X_IMU_train)
X_IMU_train_std = sc_IMU.transform(X_IMU_train)
X_IMU_val_std = sc_IMU.transform(X_IMU_val)
# X_IMU_test_std = sc_IMU.transform(X_IMU_test)

# Persist scaler
joblib.dump(sc_IMU, './models/sc_IMU.pkl')

['./models/sc_IMU.pkl']

In [11]:
print ('How many features: %d' % len(extract_feature_IMU(IMU_RO_sample_filltoprior)))

How many features: 256


In [12]:
print('Training data的比率 of IMU')
uni ,count = np.unique(y_IMU_train, return_counts=True)
print (dict(zip(uni, count)))

Training data的比率 of IMU
{0: 337, 1: 94}


In [13]:
print('Validation data的比率 of IMU')
uni ,count = np.unique(y_IMU_val, return_counts=True)
print (dict(zip(uni, count)))

Validation data的比率 of IMU
{0: 144, 1: 41}


# 丟入model 看結果(緊張..)

#### Decision Tree

In [14]:
print('======Decision Tree=======')
from sklearn.tree import DecisionTreeClassifier

# criterion : impurity function
# max_depth : maximum depth of tree
# random_state : seed of random number generator
##IMU
tree_IMU = DecisionTreeClassifier(criterion='gini', 
                              max_depth=None, 
                              random_state=4,
                                 max_features='auto')
tree_IMU.fit(X_IMU_train_std, y_IMU_train)

print('[IMU performance]')
#Validation
from sklearn.metrics import accuracy_score
y_IMU_pred = tree_IMU.predict(X_IMU_val_std)
print('Validation accuracy: %.2f' % accuracy_score(y_IMU_val, y_IMU_pred))
#Testing
# y_IMU_pred = tree_IMU.predict(X_IMU_test_std)
# print('Testing accuracy: %.2f' % accuracy_score(y_IMU_test, y_IMU_pred))

#Persist model
joblib.dump(tree_IMU, './models/tree_IMU.pkl')

======Decision Tree=======
[IMU performance]
Validation accuracy: 0.98


['./models/tree_IMU.pkl']

#### RandomForest

In [15]:
print('======Random Forest=======')
from sklearn.ensemble import RandomForestClassifier

# criterion : impurity function
# n_estimators :  number of decision trees
# random_state : seed used by the random number generator
# n_jobs : number of cores for parallelism
#IMU
forest_IMU = RandomForestClassifier(criterion='entropy',
                                max_depth=10,
                                n_estimators=500, 
                                random_state=0,
                                max_features='auto',
                                n_jobs=-1)
forest_IMU.fit(X_IMU_train_std, y_IMU_train)

print('[IMU performance]')
#Validation
y_IMU_pred = forest_IMU.predict(X_IMU_val_std)
print('Validation accuracy: %.2f' % accuracy_score(y_IMU_val, y_IMU_pred))
#Testing
# y_IMU_pred = forest_IMU.predict(X_IMU_test_std)
# print('Testing accuracy: %.2f' % accuracy_score(y_IMU_test, y_IMU_pred))


#Persist model
joblib.dump(forest_IMU, './models/forest_IMU.pkl')

======Random Forest=======
[IMU performance]
Validation accuracy: 1.00


['./models/forest_IMU.pkl']

#### KNN

In [16]:
print('======KNN=======')
from sklearn.neighbors import KNeighborsClassifier

# p=2 and metric='minkowski' means the Euclidean Distance
##IMU
knn_IMU = KNeighborsClassifier(n_neighbors=10, 
                               weights='uniform', 
                               algorithm='auto', 
                               leaf_size=5,
                               p=3, 
                               metric='minkowski', 
                               n_jobs=-1)

knn_IMU.fit(X_IMU_train_std, y_IMU_train)

print('[IMU performance]')
#Validation
y_IMU_pred = knn_IMU.predict(X_IMU_val_std)
print('Validation accuracy: %.2f' % accuracy_score(y_IMU_val, y_IMU_pred))
#Testing
# y_IMU_pred = knn_IMU.predict(X_IMU_test_std)
# print('Testing accuracy: %.2f' % accuracy_score(y_IMU_test, y_IMU_pred))


#Persist model
joblib.dump(knn_IMU, './models/knn_IMU.pkl')

======KNN=======
[IMU performance]
Validation accuracy: 0.99


['./models/knn_IMU.pkl']

#### SVM

In [17]:
print('=====SVM=======')
from sklearn.svm import SVC

# kernel: the kernel function, can be 'linear', 'poly', 'sigmoid', ...etc
# C is the hyperparameter for the error penalty term
##IMU
svm_linear_IMU = SVC(kernel='linear', C=10000.0, random_state=0, probability=True)

svm_linear_IMU.fit(X_IMU_train_std, y_IMU_train)

print('#[IMU performance]#')
#Validation
print('[Linear SVC]')
y_IMU_pred = svm_linear_IMU.predict(X_IMU_val_std)
print('Validation accuracy: %.2f' % accuracy_score(y_IMU_val, y_IMU_pred))
#Testing
# y_IMU_pred = svm_linear_IMU.predict(X_IMU_test_std)
# print('Testing accuracy: %.2f' % accuracy_score(y_IMU_test, y_IMU_pred))

##poly
svm_poly_IMU = SVC(kernel='poly', C=10000.0, degree=1, random_state=2, probability=True)

svm_poly_IMU.fit(X_IMU_train_std, y_IMU_train)

#Validation
print('[Poly SVC]')
y_IMU_pred = svm_poly_IMU.predict(X_IMU_val_std)
print('Validation accuracy: %.2f' % accuracy_score(y_IMU_val, y_IMU_pred))
#Testing
# y_IMU_pred = svm_poly_IMU.predict(X_IMU_test_std)
# print('Testing accuracy: %.2f' % accuracy_score(y_IMU_test, y_IMU_pred))

##sigmoid
svm_sigmoid_IMU = SVC(kernel='sigmoid', C=1.0, degree=100, random_state=0, probability=True)

svm_sigmoid_IMU.fit(X_IMU_train_std, y_IMU_train)

#Validation
print('[sigmoid SVC]')
y_IMU_pred = svm_sigmoid_IMU.predict(X_IMU_val_std)
print('Validation accuracy: %.2f' % accuracy_score(y_IMU_val, y_IMU_pred))
#Testing
# y_IMU_pred = svm_sigmoid_IMU.predict(X_IMU_test_std)
# print('Testing accuracy: %.2f' % accuracy_score(y_IMU_test, y_IMU_pred))


#Persist model
joblib.dump(svm_linear_IMU, './models/svm_linear_IMU.pkl')
joblib.dump(svm_poly_IMU, './models/svm_poly_IMU.pkl')
joblib.dump(svm_sigmoid_IMU, './models/svm_sigmoid_IMU.pkl')

=====SVM=======
#[IMU performance]#
[Linear SVC]
Validation accuracy: 0.99
[Poly SVC]
Validation accuracy: 0.99
[sigmoid SVC]
Validation accuracy: 0.98


['./models/svm_sigmoid_IMU.pkl']

#### Neural network

In [18]:
print('======Neural network=======')
from sklearn.neural_network import MLPClassifier

NN_IMU = MLPClassifier(solver='lbfgs', alpha=0.0001, activation='logistic', learning_rate='constant', learning_rate_init=0.001,
                    hidden_layer_sizes=(5, 2), random_state=1)

NN_IMU.fit(X_IMU_train_std, y_IMU_train)

print('#[IMU performance]#')
#Validation
y_IMU_pred = NN_IMU.predict(X_IMU_val_std)
print('Validation accuracy: %.2f' % accuracy_score(y_IMU_val, y_IMU_pred))


#Persist model
joblib.dump(NN_IMU, './models/NN_IMU.pkl')

======Neural network=======
#[IMU performance]#
Validation accuracy: 0.99


['./models/NN_IMU.pkl']

# Ensemble methods

#### Voting

In [19]:
def ensemble(X_train):
    y_pred_ensemble = []
    y_pred_ensemble.append(tree_IMU.predict(X_train))
    y_pred_ensemble.append(forest_IMU.predict(X_train))
    y_pred_ensemble.append(knn_IMU.predict(X_train))
    y_pred_ensemble.append(svm_linear_IMU.predict(X_train))
    y_pred_ensemble.append(svm_poly_IMU.predict(X_train))
    y_pred_ensemble.append(svm_sigmoid_IMU.predict(X_train))
    y_pred_ensemble.append(NN_IMU.predict(X_train))
    y_pred_ensemble = np.array(y_pred_ensemble).T
    y_pred = []
    for y_seg in y_pred_ensemble:
        y_seg = y_seg.tolist()
        if y_seg.count(0) > y_seg.count(1):
            y_pred.append(0)
        else:
            y_pred.append(1)
    #     unique, counts = numpy.unique(y_seg, return_counts=True)
    #     dict(zip(unique, counts))
    return np.array(y_pred)

In [20]:
y_IMU_pred = ensemble(X_IMU_val_std)
print('Validation accuracy: %.2f' % accuracy_score(y_IMU_val, y_IMU_pred))

Validation accuracy: 0.99


In [21]:
# from sklearn.ensemble import VotingClassifier
# from sklearn.model_selection import cross_val_score
# import itertools

# voting_IMU, best_w, best_score = None, (), -1
# for a, b, c, d, e, f, g in list(itertools.permutations(range(0, 7))):  # try some weight combination
#     voting_IMU_temp = VotingClassifier(
#                         estimators=[('tree', tree_IMU), ('forest', forest_IMU), ('knn', knn_IMU),
#                                     ('svm_linear', svm_linear_IMU), ('svm_poly', svm_poly_IMU), ('svm_rbf', svm_rbf_IMU),
#                                     ('NN', NN_IMU)],
#                           voting='soft',
#                           weights=[a, b, c, d, e, f, g])
#     scores = cross_val_score(
#         estimator=voting_IMU_temp, X=X_IMU_train_std, y=y_IMU_train, cv=10, scoring='roc_auc')
#     print('%s: %.3f (+/- %.3f)' % ((a, b, c, d, e, f, g), scores.mean(), scores.std()))
#     if best_score < scores.mean():
#         voting_IMU, best_w, best_score = voting_IMU_temp, (a, b, c, d, e, f, g), scores.mean()

# print('\nBest %s: %.3f' % (best_w, best_score))

# #Persist model
# joblib.dump(voting_IMU, './models/Voting_IMU.pkl')

# Simulating real world

### IMU

In [22]:
import random

X_combined_IMU_All = []
y_combined_IMU_All = []

numberOfSampleAll = 30
##先給0~X Other then replace RO in random place
for random_Other_sample in random.sample(IMU_Other_sampleAll, numberOfSampleAll):
    X_combined_IMU_All.append(random_Other_sample)
    y_combined_IMU_All.append(0)


insertROcount = 10
for index, random_RO_sample in zip(np.random.randint(numberOfSampleAll, size=insertROcount), random.sample(IMU_RO_sampleAll, insertROcount)):
    X_combined_IMU_All[index] = random_RO_sample
    y_combined_IMU_All[index] = 1
    
##flatten
X_combined_IMU_fla = []
y_combined_IMU_fla = []

for s, c in zip(X_combined_IMU_All, y_combined_IMU_All):
    X_combined_IMU_fla.extend(s.tolist())
    listofzerosOrOne = [c] * s.shape[0]
    y_combined_IMU_fla.extend(listofzerosOrOne)
    
##Segmentation(by windows Size, and Overlapping= windows size - 1 sec)
overlapping = windowSize_sec - 1
X_combined_IMU_seg = [] 

for start in range(0, len(X_combined_IMU_fla), windowSize_sec*sampleRate_IMU_acc - overlapping*sampleRate_IMU_acc):
    end = start + sampleRate_IMU_acc*windowSize_sec
    if len(X_combined_IMU_fla[start:end]) < sampleRate_IMU_acc*windowSize_sec:
        continue
    else:
        X_combined_IMU_seg.append(X_combined_IMU_fla[start:end])
        
# ##feature extraction
X_combined_IMU = []

for X_combined_IMU_seg_sample in X_combined_IMU_seg:
    X_combined_IMU.append(extract_feature_IMU(np.array(X_combined_IMU_seg_sample)))
        
##Normalization
X_combined_IMU_std = sc_IMU.transform(X_combined_IMU)

## Input to models

In [23]:
#將Pred結果展開成sample比較 #Overlapping 實際執行秒數 = (Sec*Num)-[(Num-1)*Overlap]
def pred_flatToSamples_IMU(y_pred):
    allofSec = windowSize_sec*len(y_pred)-(len(y_pred)-1)*overlapping
    y_pred_sample = [0] * (allofSec*sampleRate_IMU_acc)
    for pred, index in zip(y_pred, range(0, allofSec, windowSize_sec-overlapping)):
        if pred == 1:
            start = index*sampleRate_IMU_acc
            end = start + sampleRate_IMU_acc*windowSize_sec
            listofOne = [1] * (sampleRate_IMU_acc*windowSize_sec)
            y_pred_sample[start:end] = listofOne
    return y_pred_sample

In [24]:
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

print('--IMU--')
tree_IMU = joblib.load('./models/tree_IMU.pkl')
y_pred = tree_IMU.predict(X_combined_IMU_std)
y_pred_sample = pred_flatToSamples_IMU(y_pred)

# plot(np.array(y_combined_IMU_fla[:len(y_pred_sample)])[:,np.newaxis])
# plot(np.array(y_pred_sample)[:,np.newaxis])
dist, path = fastdtw(np.array(y_combined_IMU_fla[:len(y_pred_sample)])[:,np.newaxis], 
                            np.array(y_pred_sample)[:,np.newaxis], 
                            dist=euclidean)
print('Decision tree \n-DTW:',dist)
print('-Accuracy: %.2f' % accuracy_score(y_combined_IMU_fla[:len(y_pred_sample)], y_pred_sample))

forest_IMU = joblib.load('./models/forest_IMU.pkl')
y_pred = forest_IMU.predict(X_combined_IMU_std)
y_pred_sample = pred_flatToSamples_IMU(y_pred)

dist, path = fastdtw(np.array(y_combined_IMU_fla[:len(y_pred_sample)])[:,np.newaxis], 
                            np.array(y_pred_sample)[:,np.newaxis], 
                            dist=euclidean)
print('Random forest \n-DTW:',dist)
print('-Accuracy: %.2f' % accuracy_score(y_combined_IMU_fla[:len(y_pred_sample)], y_pred_sample))

knn_IMU = joblib.load('./models/knn_IMU.pkl')
y_pred = knn_IMU.predict(X_combined_IMU_std)
y_pred_sample = pred_flatToSamples_IMU(y_pred)

dist, path = fastdtw(np.array(y_combined_IMU_fla[:len(y_pred_sample)])[:,np.newaxis], 
                            np.array(y_pred_sample)[:,np.newaxis], 
                            dist=euclidean)
print('KNN \n-DTW:',dist)
print('-Accuracy: %.2f' % accuracy_score(y_combined_IMU_fla[:len(y_pred_sample)], y_pred_sample))

svm_linear_IMU = joblib.load('./models/svm_linear_IMU.pkl')
y_pred = svm_linear_IMU.predict(X_combined_IMU_std)
y_pred_sample = pred_flatToSamples_IMU(y_pred)

dist, path = fastdtw(np.array(y_combined_IMU_fla[:len(y_pred_sample)])[:,np.newaxis], 
                            np.array(y_pred_sample)[:,np.newaxis], 
                            dist=euclidean)
print('SVM linear \n-DTW:',dist)
print('-Accuracy: %.2f' % accuracy_score(y_combined_IMU_fla[:len(y_pred_sample)], y_pred_sample))

svm_poly_IMU = joblib.load('./models/svm_poly_IMU.pkl')
y_pred = svm_poly_IMU.predict(X_combined_IMU_std)
y_pred_sample = pred_flatToSamples_IMU(y_pred)

dist, path = fastdtw(np.array(y_combined_IMU_fla[:len(y_pred_sample)])[:,np.newaxis], 
                            np.array(y_pred_sample)[:,np.newaxis], 
                            dist=euclidean)
print('SVM poly \n-DTW:',dist)
print('-Accuracy: %.2f' % accuracy_score(y_combined_IMU_fla[:len(y_pred_sample)], y_pred_sample))

svm_sigmoid_IMU = joblib.load('./models/svm_sigmoid_IMU.pkl')
y_pred = svm_sigmoid_IMU.predict(X_combined_IMU_std)
y_pred_sample = pred_flatToSamples_IMU(y_pred)

dist, path = fastdtw(np.array(y_combined_IMU_fla[:len(y_pred_sample)])[:,np.newaxis], 
                            np.array(y_pred_sample)[:,np.newaxis], 
                            dist=euclidean)
print('SVM sigmoid \n-DTW:',dist)
print('-Accuracy: %.2f' % accuracy_score(y_combined_IMU_fla[:len(y_pred_sample)], y_pred_sample))

NN_IMU = joblib.load('./models/NN_IMU.pkl')
y_pred = NN_IMU.predict(X_combined_IMU_std)
y_pred_sample = pred_flatToSamples_IMU(y_pred)

dist, path = fastdtw(np.array(y_combined_IMU_fla[:len(y_pred_sample)])[:,np.newaxis], 
                            np.array(y_pred_sample)[:,np.newaxis], 
                            dist=euclidean)
print('NN \n-DTW:',dist)
print('-Accuracy: %.2f' % accuracy_score(y_combined_IMU_fla[:len(y_pred_sample)], y_pred_sample))

--IMU--
Decision tree 
-DTW: 0.0
-Accuracy: 0.87
Random forest 
-DTW: 0.0
-Accuracy: 0.90
KNN 
-DTW: 0.0
-Accuracy: 0.90
SVM linear 
-DTW: 0.0
-Accuracy: 0.88
SVM poly 
-DTW: 0.0
-Accuracy: 0.88
SVM sigmoid 
-DTW: 289.0
-Accuracy: 0.78
NN 
-DTW: 0.0
-Accuracy: 0.89


## Ensemble

In [25]:
y_pred = ensemble(X_combined_IMU_std)
y_pred_sample = pred_flatToSamples_IMU(y_pred)

dist, path = fastdtw(np.array(y_combined_IMU_fla[:len(y_pred_sample)])[:,np.newaxis], 
                            np.array(y_pred_sample)[:,np.newaxis], 
                            dist=euclidean)
print('Ensembnle \n-DTW:',dist)
print('-Accuracy: %.2f' % accuracy_score(y_combined_IMU_fla[:len(y_pred_sample)], y_pred_sample))

Ensembnle 
-DTW: 0.0
-Accuracy: 0.89


In [26]:
1/0

ZeroDivisionError: division by zero

# Experiencing real data

IMU

In [ ]:
IMU_rW_pd = pd.read_csv('D:/GoogleDrive/Graduate/Research/Research_HsinWei/Programs/data/realWorldData/raw_data_RW01.csv', header=None)
# IMU_rW_pd = pd.read_csv('D:/GoogleDrive/Graduate/Research/Research_HsinWei/Programs/data/IMUData/ReachOut/HW_raw_acc_RO_02.csv', header=None)

IMU_rW_pd.columns = ['eaccx', 'eaccy', 'eaccz', 'accx', 'accy', 'accz', 'gyrox', 'gyroy', 'gyroz', 'timestamp']

IMU_rW = IMU_rW_pd[['accx', 'accy', 'accz', 'gyrox', 'gyroy', 'gyroz']].values.tolist()

##Segmentation(by windows Size, and Overlapping= windows size - 1 sec)
overlapping = windowSize_sec - 1
X_rW_IMU_seg = []

for start in range(0, len(IMU_rW), windowSize_sec*sampleRate_IMU_acc - overlapping*sampleRate_IMU_acc):
    end = start + sampleRate_IMU_acc*windowSize_sec
    if len(IMU_rW[start:end]) < sampleRate_IMU_acc*windowSize_sec:
        break
    else:
        X_rW_IMU_seg.append(IMU_rW[start:end])
        
#Preprocessing
X_rW_IMU_seg_conv = []
for X_rW_IMU_seg_sam in X_rW_IMU_seg:
    X_rW_IMU_seg_conv.append(X_rW_IMU_seg_sam)
        
##feature extraction
X_rW_IMU = []
for X_rW_IMU_seg_sample in X_rW_IMU_seg_conv:
    X_rW_IMU.append(extract_feature_IMU(np.array(X_rW_IMU_seg_sample)))
        
##Normalization
X_rW_IMU_std = sc_IMU.transform(X_rW_IMU)

Kinect

In [ ]:
Kinect_rW_pd = pd.read_csv('D:/GoogleDrive/Graduate/Research/Research_HsinWei/Programs/data/realWorldData/VSFile_RW01.csv', header=None)
Kinect_rW_pd.columns = ['WristRight_x', 'WristRight_y', 'WristRight_z', 
                  'WristLeft_x', 'WristLeft_y', 'WristLeft_z', 
                  'ElbowRight_x', 'ElbowRight_y', 'ElbowRight_z',
                  'ElbowLeft_x', 'ElbowLeft_y', 'ElbowLeft_z',
                  'ShoulderRight_x', 'ShoulderRight_y', 'ShoulderRight_z',
                  'ShoulderLeft_x', 'ShoulderLeft_y', 'ShoulderLeft_z',
                  'SkeletonID', 'Head_image_x', 'Head_image_y', 'timestamp']

Kinect_rW_pd_ntime = Kinect_rW_pd[['WristRight_x', 'WristRight_y', 'WristRight_z', 
                  'WristLeft_x', 'WristLeft_y', 'WristLeft_z', 
                  'ElbowRight_x', 'ElbowRight_y', 'ElbowRight_z',
                  'ElbowLeft_x', 'ElbowLeft_y', 'ElbowLeft_z',
                  'ShoulderRight_x', 'ShoulderRight_y', 'ShoulderRight_z',
                  'ShoulderLeft_x', 'ShoulderLeft_y', 'ShoulderLeft_z',
                  'SkeletonID', 'Head_image_x', 'Head_image_y']]

Kinect_rW = Kinect_rW_pd_ntime.values.tolist()

### Feed into models

In [ ]:
y_IMU_pred = ensemble(X_rW_IMU_std)
print(y_IMU_pred)
IMU_sample_rw_pred = pred_flatToSamples_IMU(y_IMU_pred)

# #Aligning
# from datetime import datetime
# IMU_rW_time = IMU_rW_pd['timestamp']
# Kinect_rW_time = Kinect_rW_pd['timestamp']

# startIndex_IMU = 0
# start_Kinect = datetime.strptime(Kinect_rW_time[0], '%H:%M:%S:%f')
# IMU_rW_time = pd.to_datetime(IMU_rW_time, format='%H:%M:%S:%f')
# for index, IMU_timestamp in zip(range(0, len(IMU_rW_time)), IMU_rW_time):
#     if IMU_timestamp < start_Kinect:
#         continue
#     else:
#         startIndex_IMU = index
#         break
        
# IMU_sample_rw_pred = IMU_sample_rw_pred[startIndex_IMU:]

Sign IMU result in Kinect video

In [ ]:
#Sign IMU result in Kinect video
IMU_result_inKinect = []

IMU_rW_time = pd.to_datetime(IMU_rW_pd['timestamp'], format='%H:%M:%S:%f')
Kinect_rW_time = pd.to_datetime(Kinect_rW_pd['timestamp'], format='%H:%M:%S:%f')
i = 0
for Kinect_rW_timestamp in Kinect_rW_time:
    for si in range(i, len(IMU_rW_time)):
        if IMU_rW_time[si] <= Kinect_rW_timestamp and Kinect_rW_timestamp < IMU_rW_time[si+1]:
            IMU_result_inKinect.append(IMU_sample_rw_pred[i])
            i = si
            break
            
# IMU_Kinect_sample_rw_pred = pd.DataFrame(np.array([IMU_result_inKinect, Kinect_sample_rw_pred]).T)

### Save to csv include result of predicting, headX_video, headY_video

In [ ]:
result = pd.concat([pd.Series(IMU_result_inKinect), Kinect_rW_pd[['Head_image_x', 'Head_image_y', 'timestamp']]], ignore_index=True, axis=1)

result.to_csv('D:/GoogleDrive/Graduate/Research/Research_HsinWei/Programs/data/realWorldData/ReachOutResult.csv', header=None, index=False)

# Next Step

- 測試真實的testing data看效果如何
- IMU加更多的feature..(切更細的mean(O), Correlation, Covariance..)
- Kinect加angle相關的atrribute, 並且座標zero點在自己身上,不會依據距離遠近而誤判
- Kinect & IMU feature 合併的話??不是個別判斷
- 程式部分
    - 將extract feature寫成function(O)
    - set sample/sec to var

DONE
- 實驗重作...(O)
- 程式部分
    - 將extract feature寫成function(O)
    - set sample/sec to var(O)

# Save to csv include percentage, headX_video, headY_video

In [ ]:
result = pd.concat([percentage, df[['HeadX_Video', 'HeadY_Video', 'TimeStamp']]], axis=1)

result.to_csv('C:\Research_temp\data\Result\ReachOutResult.csv', header=None, index=False)

In [ ]:
import glob
import os

path = r'D:\GoogleDrive\Graduate\Research\Research_HsinWei\Programs\data\IMUData\ReachOut'                     # use your path
all_files = glob.glob(os.path.join(path, "*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent


In [ ]:
for a, b in range(10), range(11):
    print(a, b)